### Урок 3. Классификация. Логистическая регрессия.

#### 1. *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

Как один из возможных вариантов можно предложиит заменять предсказанные нулевые значения, в которых логарифм будет не определен, на очень маленькое значение, мало отличное от 0, например 1-е17. 

In [2]:
import numpy as np

In [17]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [9]:
def calc_logloss_1(y, y_pred):
    y_pred = np.where(y_pred!=0, y_pred, 1e-17)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [11]:
# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0.8, 0.1])
calc_logloss_1(y1, y_pred1)

0.164252033486018

In [13]:
# Плохой пример применения
y1 = np.array([1, 0, 1, 0])
y_pred1 = np.array([0.99, 0, 0, 0])
calc_logloss_1(y1, y_pred1)

9.78849922918807

Или, как еще вариант, в цикле проверять значения y и y_pred на равентсво y_pred = 0 и отбрасывать такие пары значений перед передачей их в функцию. 

#### 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [3]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [4]:
X

array([[1.00e+00, 1.00e+00, 5.00e+02, 1.00e+00],
       [1.00e+00, 1.00e+00, 7.00e+02, 1.00e+00],
       [1.00e+00, 2.00e+00, 7.50e+02, 2.00e+00],
       [1.00e+00, 5.00e+00, 6.00e+02, 1.00e+00],
       [1.00e+00, 3.00e+00, 1.45e+03, 2.00e+00],
       [1.00e+00, 0.00e+00, 8.00e+02, 1.00e+00],
       [1.00e+00, 5.00e+00, 1.50e+03, 3.00e+00],
       [1.00e+00, 1.00e+01, 2.00e+03, 3.00e+00],
       [1.00e+00, 1.00e+00, 4.50e+02, 1.00e+00],
       [1.00e+00, 2.00e+00, 1.00e+03, 2.00e+00]])

In [5]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

In [6]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [7]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [8]:
X_st

array([[ 1.        ,  1.        , -0.97958969,  1.        ],
       [ 1.        ,  1.        , -0.56713087,  1.        ],
       [ 1.        ,  2.        , -0.46401617,  2.        ],
       [ 1.        ,  5.        , -0.77336028,  1.        ],
       [ 1.        ,  3.        ,  0.97958969,  2.        ],
       [ 1.        ,  0.        , -0.36090146,  1.        ],
       [ 1.        ,  5.        ,  1.08270439,  3.        ],
       [ 1.        , 10.        ,  2.11385144,  3.        ],
       [ 1.        ,  1.        , -1.08270439,  1.        ],
       [ 1.        ,  2.        ,  0.05155735,  2.        ]])

In [15]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [18]:
etas = [1e-1, 5e-2, 1e-2, 5e-3, 1e-3, 5e-4, 1e-4]
n_iters = [250, 500, 800, 1000, 5000, 10000]
np.random.seed(42)

best_error = np.inf
best_params = {}


for eta in etas:
    for n_iter in n_iters:
        W = np.random.randn(X_st.shape[1])
        n = X_st.shape[0]
        curr_eta = eta
        for i in range(n_iter):
            z = np.dot(X_st, W)
            y_pred = sigmoid(z)
            err = calc_logloss(y, y_pred)
            dQ = 1/n * X_st.T @ (y_pred - y)
            W -= curr_eta * dQ

            if i % 10 == 0:
                curr_eta /= 1.1
                                
        if err < best_error:
            best_error = err
            best_params = {
                'eta': eta,
                'n_iter': n_iter
            }
print(f'Лучшая ошибка {best_error} с параметрами {best_params}')

Лучшая ошибка 0.4610463395426619 с параметрами {'eta': 0.05, 'n_iter': 10000}


#### 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [21]:
def eval_model(X_st, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X_st.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X_st, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X_st.T @ (y_pred - y)
        W -= eta * dQ
#         if i % (iterations / 10) == 0:
#             print(i, W, err)
    return W

In [22]:
def calc_pred_proba(X, W):
    return sigmoid(np.dot(X, W))

In [23]:
W = eval_model(X_st, y, iterations=10000, eta=0.05)
p = calc_pred_proba(X_st, W)
p

array([0.33411971, 0.22116833, 0.98222268, 0.00394414, 0.70685345,
       0.40031129, 0.99338372, 0.10829936, 0.36649211, 0.96443976])

#### 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [24]:
def calc_pred(X, W):
    
    m = X.shape[0]
    
    y_predicted = np.zeros(m)

    A = np.squeeze(sigmoid(np.dot(X, W)))

    # За порог отнесения к тому или иному классу примем вероятность 0.5
    for i in range(A.shape[0]):
        if (A[i] > 0.5): 
            y_predicted[i] = 1
        elif (A[i] <= 0.5):
            y_predicted[i] = 0

    return y_predicted

In [25]:
y_preds = calc_pred(X_st, W)

print(f'y_pred  {y_preds}')
print(f'y       {y}')
    

y_pred  [0. 0. 1. 0. 1. 0. 1. 0. 0. 1.]
y       [0. 0. 1. 0. 1. 0. 1. 0. 1. 1.]


#### 5. *Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

In [26]:
error_matrix = np.zeros((2, 2))
for i in range(len(y)):
    if y_preds[i] == 1 and y[i] != 0:
        error_matrix[0][0] += 1
    elif y_preds[i] == 1 and y[i] != 1:
        error_matrix[0][1] += 1
    elif y_preds[i] == 0 and y[i] != 1:
        error_matrix[1][1] += 1
    else:
        error_matrix[1][0] += 1
error_matrix

array([[4., 0.],
       [1., 5.]])

In [27]:
def accuracy(y_pred, y):
    all = 0
    for i in range(len(y)):
        if y[i] == y_pred[i]:
            all +=1
    acc = all / len(y)
    return acc


In [28]:
accuracy(y_preds, y)

0.9

In [29]:
def precision(error_matrix):
    pr = error_matrix[0][0] / (error_matrix[0][0] + error_matrix[0][1])
    return pr

In [30]:
precision(error_matrix)

1.0

In [31]:
def recall(error_matrix):
    re = error_matrix[0][0] / (error_matrix[0][0] + error_matrix[1][0])
    return re

In [32]:
recall(error_matrix)

0.8

In [33]:
def F_score(error_matrix):
    f_s = round(2 * precision(error_matrix) * recall(error_matrix) / (recall(error_matrix) + precision(error_matrix)), 2)
    return f_s

In [34]:
F_score(error_matrix)

0.89

#### 6. Могла ли модель переобучиться? Почему?

Могла бы, если продолжить изменять eta и iterations. При ручном подборе параметров было замечено, что при дальнейшем изменении параметров и при уменьшении logloss веса начинали расти, что свидетельствовало о переобучении модели.